# PetFinder
kaggleコンペpetfinder　https://www.kaggle.com/c/petfinder-adoption-prediction
をfastaiやpytorch Lightningのような上位の
ライブラリなしで実装したノートブック


## Configとwandb準備

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import sys
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm

from torch.cuda.amp import autocast, GradScaler

import warnings
warnings.filterwarnings('ignore')
torch.backends.cudnn.benchmark = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
CFG = {
    "save_prev"     : [True, ["petfinder_*.pth"]],
    "seed"          : 42,
    'device'        : "cuda:0" if torch.cuda.is_available() else "cpu",
    "input_img"     : '../input/petfinder-pawpularity-score/train/',
    "input_path"    : '../input/petfinder-smogn-dataset/train_drop_duplicated.csv',
    "output_path"   : './',
    "pretrain"      : "../input/petfinder-fastai-semisupervised-models/petfinder_swin_binary_ss_fastai_smogn_4.pth",
    "save_name"     : "petfinder_swin_binary_fastai_ss_smogn",
    "model"         : "swin_large_patch4_window7_224_in22k",
    "loss"          : "binary",
    "size"          : 224,
    "fold"          : 4,
    "batch_size"    : 32,
    "epochs"        : 5,
    "n_data"        : None,  # Batch * step
    "mixup_ratio"   : 0,
    "lr"            : 1e-6,
    "accum_iter"    : 2,
    'early_stopping': 3,
    'verbose_step'  : 1,
    "num_workers"   : 4
}

CFG

## データロード

In [ ]:
print("chiba")

## training

## inference